In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 30    # steepness factor
num_blocks = 1    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=10)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [09:30<00:00,  1.75it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      1.583312               0.0      0.569553
std       2.198417               0.0      0.330276
min       0.000007               0.0      0.090976
25%       0.125648               0.0      0.329740
50%       0.648800               0.0      0.627853
75%       2.039892               0.0      0.791127
max      11.392356               0.0      7.891120
Number of infeasible solution: 0


## Heuristic

In [8]:
model_heur = model.first_solution_heuristic()

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    model_heur.set_param_val({"p":p, "a":a})
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    params.append(list(p)+list(a))
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_e.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:24<00:00, 11.81it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean     68.829961               0.0      0.084098
std      42.395181               0.0      0.038278
min       4.334028               0.0      0.058672
25%      32.603227               0.0      0.062121
50%      68.051118               0.0      0.075356
75%      90.896112               0.0      0.078228
max     188.935700               0.0      0.328066
Number of infeasible solution: 0


## Learnable Rounding

In [10]:
# hyperparameters
penalty_weight = 10   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-2             # learning rate
batch_size = 64       # batch size

In [11]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# define Rosenbrock objective functions and constraints for both problem types
obj_rel, constrs_rel = nmRosenbrock(["x"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
obj_rnd, constrs_rnd = nmRosenbrock(["x_rnd"], ["p", "a"], steepness, num_blocks, penalty_weight=penalty_weight)
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],
                       output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = [smap, rnd]
loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
problem = nm.problem.Problem(components, loss)

In [12]:
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(problem.parameters(), lr=lr)
# create a trainer for the problem
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test, optimizer, 
                            epochs=epochs, patience=patience, warmup=warmup)
# training for the rounding problem
best_model = trainer.train()
# load best model dict
problem.load_state_dict(best_model)

epoch: 0  train_loss: 20.87644386291504
epoch: 1  train_loss: 12.032898902893066
epoch: 2  train_loss: 9.679839134216309
epoch: 3  train_loss: 8.76450252532959
epoch: 4  train_loss: 9.432135581970215
epoch: 5  train_loss: 8.903563499450684
epoch: 6  train_loss: 8.251978874206543
epoch: 7  train_loss: 8.442489624023438
epoch: 8  train_loss: 7.854027271270752
epoch: 9  train_loss: 5.407695293426514
epoch: 10  train_loss: 5.0392961502075195
epoch: 11  train_loss: 4.812459945678711
epoch: 12  train_loss: 4.8394598960876465
epoch: 13  train_loss: 4.972204208374023
epoch: 14  train_loss: 4.737155437469482
epoch: 15  train_loss: 4.713879108428955
epoch: 16  train_loss: 4.8075127601623535
epoch: 17  train_loss: 4.861983776092529
epoch: 18  train_loss: 4.9503936767578125
epoch: 19  train_loss: 4.603105545043945
epoch: 20  train_loss: 4.865034580230713
epoch: 21  train_loss: 4.891582012176514
epoch: 22  train_loss: 4.606245040893555
epoch: 23  train_loss: 4.912778854370117
epoch: 24  train_loss:

<All keys matched successfully>

In [13]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = output["test_x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_nm_e.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 373.20it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.931294          0.000678      0.002333
std       4.396258          0.008713      0.000758
min       0.001208          0.000000      0.000999
25%       1.388503          0.000000      0.002000
50%       3.607770          0.000000      0.002006
75%       7.454648          0.000000      0.002575
max      21.020573          0.148556      0.012954
Number of infeasible solution: 9
